### Deploy the Stable Diffusion 2 Inpainting model to a SageMaker endpoint
Following similar steps, you can deploy other pre-trained models from JumpStart. Such as the **Stable Diffusion 2 Inpainting** model from [Stability AI](https://stability.ai/blog/stable-diffusion-public-release). It takes an image, a mask image and a text prompt as input. It replaces the mask area of the original image with an image described by the text prompt to generate a new image. Follow the same steps from the above and deploy this model from the JumpStart model hub to a SageMaker endpoint (with instance type `ml.g5.2xlarge`). Once the model is successfully deployed, you can use below code to invoke the endpoint.

### Deploy a pre-trained Stable Diffusion model from the SageMaker JumpStart console
In the navigation pane, under **SageMaker JumpStart**, choose **Model, notebooks, solutions**. You’re presented with a range of solutions, foundation models, and other artifacts that can help you get started with a specific model or a specific business problem or use case. If you want to experiment in a particular area, you can use the search function. Or you can simply browse the artifacts to find the relevant model or business solution for your needs. To start exploring the Stable Diffusion models, complete the following steps:

1. Go to the `Foundation Models` section and select the **Stable Diffusion 2 Inpainting** model and click **View model**.
<div>
    <img src="./img/a1a.png" alt="Image jumpstart" width="1000" style="display:inline-block">
</div>
<br>

2. A new tab is opened with the options to train, deploy and view model details as shown below.
<br><br>
3. In the Deploy Model section, click the **Deploy** button.
<br>
<div>
    <img src="./img/a2a.png" alt="Image sb2.1" width="1000" style="display:inline-block">
</div>
<br>



The deploy action will start a new tab showing the model creation status and the model deployment status.
<div>
    <img src="./img/3a.png" alt="Image preparemodel" width="500" style="display:inline-block">
</div>
<br>
<br>
Wait until the Endpoint Status changes to "In Service" 
<br>
<br>
<div>
    <img src="./img/4a.png" alt="Image createendpoint" width="600" style="display:inline-block">
</div>
<br>

4. When the endpoint is deployed, choose **Open Notebook** to open a Jupyter notebook with Python code. Or use the code at the end of this section to invoke the endpoint.

<div>
    <img src="./img/4aa.png" alt="Image opennotebook" width="700" style="display:inline-block">
</div>
<br>



You will be prompted to Select a Notebook Envrionment, accept the default
<br>
<br>Image : Data Science 2.0
<br>Kernal : Python 3
<br>Instance Type : ml.t3.medium

<div>
    <img src="./img/5a.png" alt="Image opennotebook" width="700" style="display:inline-block">
</div>
<br>

In [ ]:
import boto3
import matplotlib.pyplot as plt
import numpy as np
import json
import base64
from PIL import Image
from io import BytesIO
endpoint_name = 'jumpstart-dft-stable-diffusion-2-inpainting'
input_img_file_name = "dog_suit.jpg"
input_img_mask = "dog_suit_mask.jpg"

In [ ]:
def encode_img(img_name):
    with open(img_name,'rb') as f: img_bytes = f.read()
    encoded_img = base64.b64encode(bytearray(img_bytes)).decode()
    return encoded_img

encoded_input_image = encode_img(input_img_file_name)
encoded_mask_image = encode_img(input_img_mask)


payload = { "prompt":"a white cat, blue eyes, wearing a sweater, lying in park", "image": encoded_input_image, "mask_image":encoded_mask_image, "num_inference_steps":50, "guidance_scale":7.5, "seed": 1}

def query_endpoint(payload):
    """query the endpoint with the json payload encoded in utf-8 format."""
    encoded_payload = json.dumps(payload).encode('utf-8')
    client = boto3.client('runtime.sagemaker')
    # Accept = 'application/json;jpeg' returns the jpeg image as bytes encoded by base64.b64 encoding.
    # To receive raw image with rgb value set Accept = 'application/json'
    # To send raw image, you can set content_type = 'application/json' and encoded_image as np.array(PIL.Image.open('low_res_image.jpg')).tolist()
    # Note that sending or receiving payload with raw/rgb values may hit default limits for the input payload and the response size.
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json;jpeg', Accept = 'application/json;jpeg', Body=encoded_payload)
    return response

def display_image(img, title):
    plt.figure(figsize=(12,12))
    plt.imshow(np.array(img))
    plt.axis('off')
    plt.title(title)
    plt.show()

def parse_and_display_response(query_response):
    """Parse the endpoint response and display the generated images"""
    
    response_dict = json.loads(query_response['Body'].read())
    generated_images = response_dict['generated_images']
    
    for generated_image in generated_images:
        with BytesIO(base64.b64decode(generated_image.encode())) as generated_image_decoded:
            with Image.open(generated_image_decoded) as generated_image_np:
                generated_image_rgb = generated_image_np.convert("RGB")
                display_image(generated_image_rgb, "Inpainted Image")

In [ ]:
Image.open("dog_suit.jpg")

In [ ]:
Image.open("dog_suit_mask.jpg")

In [ ]:
payload = { 
    "prompt":"a white cat, blue eyes, wearing a sweater, lying in park",
    "image":encoded_input_image, 
    "mask_image":encoded_mask_image, 
    "num_inference_steps":30,
    "guidance_scale":7.5,
    "num_images_per_prompt":2,
    "seed": 1,
    "negative_prompt": "poorly drawn feet",
    "batch_size":2
}
query_response = query_endpoint(payload)
parse_and_display_response(query_response)

## Clean up

Before we move on, don’t forget to delete your endpoints when you’re finished. On the previous tab, under **Delete Endpoint**, choose **Delete**. Do the same to other endpoints that you have created during the lab.

<div>
    <img src="./img/delete.png" alt="Image delete" width="800" style="display:inline-block">
</div>
<br>